## 0. Libarary 불러오기 및 경로설정

In [26]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import cv2

from loss import F1Loss, FocalLoss
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
from torchvision.transforms.functional import to_pil_image # tensor to pil_image
import torchvision.models as models

In [32]:
# 데이터셋 폴더 경로를 지정
train_dir = '../../../../input/data/train'
test_dir = '../../../../input/data/eval'
model_dir = '../../../../code/models'

In [3]:
train_csv = pd.read_csv(os.path.join(train_dir, 'train.csv'))
train_csv.tail(3)

,id,gender,race,age,path
2697,006956,male,Asian,19,006956_male_Asian_19
2698,006957,male,Asian,20,006957_male_Asian_20
2699,006959,male,Asian,19,006959_male_Asian_19


In [4]:
print(f"CUDA: {torch.cuda.is_available()}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

CUDA: True


In [5]:
mapping_gender = lambda x: int(x == "female")
mapping_age = lambda x: min(2, x // 30)
mapping_mask = lambda x: x.startswith('mask') and 0 or (x.startswith('incorrect') and 1 or 2)

In [6]:
# pre-processing
# path to image_path 변경
image_dir = os.path.join(train_dir, 'images')

train_meta = pd.DataFrame()
for i in range(len(train_csv)):
    _, gender, _, age, path = train_csv.iloc[i]
    image_path = os.path.join(image_dir, path)

    li = []
    for f in os.listdir(image_path):
        if not f.startswith('.'):
            target = 6*mapping_gender(gender) + 3*mapping_age(age) + mapping_mask(f)
            li.append((target, os.path.join(image_path, f)))
    li = pd.DataFrame({name: data for name, data in  zip(['y', 'path'], zip(*li))})    
    train_meta = train_meta.append(li, ignore_index = True) # 뒤에 계속 합쳐두기
train_meta.tail(3)

,y,path
18897,2,../../../../input/data/train/images/006959_mal...
18898,1,../../../../input/data/train/images/006959_mal...
18899,2,../../../../input/data/train/images/006959_mal...


In [7]:
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])

In [8]:
# train, test dataset 나누기
X_train, X_eval, y_train, y_eval = train_test_split(
        train_meta.path.to_numpy(), 
        train_meta.y.to_numpy(), 
        test_size=0.2, shuffle=False)

print(X_train.shape, y_train.shape)
print(X_eval.shape, y_eval.shape)

(15120,) (15120,)
(3780,) (3780,)


In [10]:
class TrainDataset(Dataset):
    # input: image_list, target_list
    def __init__(self, X, y, transform=None):
        self.image_paths = X
        self.target = y
        self.transform = transform
    
    # output: PIL_image, label
    def __getitem__(self, index):
        images = []
        image = Image.open(self.image_paths[index])
        if self.transform:
            image = self.transform(image)
        
        return (image, self.target[index])
    
    def __len__(self):
        return len(self.image_paths)

In [11]:
train_ds = TrainDataset(X_train, y_train, transform)
eval_ds = TrainDataset(X_eval, y_eval, transform)

len(train_ds), len(eval_ds)

(15120, 3780)

In [12]:
BATCH_SIZE = 8
train_loader = DataLoader(
    train_ds,
    batch_size = BATCH_SIZE,
    shuffle=True
)
eval_loader = DataLoader(
    eval_ds,
    batch_size = BATCH_SIZE,
    shuffle=True
)

len(train_loader)*BATCH_SIZE, len(eval_loader)*BATCH_SIZE

(15120, 3784)

In [13]:
# EDA
def show_images(data, n = 5, rows=1, cols=7):
    plt.figure(figsize=(20,14))

    k = 1
    for im, *ans in data[:n]:
        plt.subplot(rows, cols, k)
        plt.imshow(to_pil_image(im))
        plt.title(ans, fontsize = 16)
        plt.axis('off')
        k += 1
    plt.show()

In [14]:
# images = []
# for i in range(1):
#     data = train_loader
#     images.append(data)


In [15]:
# plt.figure(figsize=(20,4))

# for i, x in enumerate(['mask', 'gender', 'age', 'y']):
#     data = train_meta[x].value_counts()
#     plt.subplot(1, 4, i+1)
#     plt.bar(data.index, data.values, tick_label=data.index)
#     plt.title(x);

## 1. Model 정의

In [17]:
lr = 0.05
model = models.resnet34(pretrained=True).to(device)
loss_fn = nn.CrossEntropyLoss()
optm = torch.optim.Adam(model.parameters(), lr=lr)

num_classes = 18
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes).to(device)

# for x in model.children():
#     print(x)

In [20]:
model.train()
for epoch in range(3):
    # image shape : [8, 3, 512, 384]
    for i, (imgs, labels) in enumerate(train_loader):
        imgs = Variable(imgs).to(device)
        labels = Variable(labels).to(device)
        
        y_pred = model(imgs)
        loss = loss_fn(y_pred, labels)

        optm.zero_grad()
        loss.backward()
        optm.step()
        
        if i % 30 == 0:
            print("epoch: {} Loss: {:.4f}".format(epoch, loss.data))

print('done!')

epoch: 0 Loss: 1.3554
epoch: 0 Loss: 2.5032
epoch: 0 Loss: 1.4187
epoch: 0 Loss: 2.3878
epoch: 0 Loss: 2.2375
epoch: 0 Loss: 1.3378
epoch: 0 Loss: 1.7886
epoch: 0 Loss: 1.1882
epoch: 0 Loss: 1.2984
epoch: 0 Loss: 1.0064
epoch: 0 Loss: 2.0053
epoch: 0 Loss: 0.8394
epoch: 0 Loss: 1.0736
epoch: 0 Loss: 1.4622
epoch: 0 Loss: 1.1537
epoch: 0 Loss: 1.4220
epoch: 0 Loss: 2.4918
epoch: 0 Loss: 0.9883
epoch: 0 Loss: 1.4397
epoch: 0 Loss: 1.0044
epoch: 0 Loss: 1.2077
epoch: 0 Loss: 1.6809
epoch: 0 Loss: 1.5316
epoch: 0 Loss: 2.5592
epoch: 0 Loss: 1.0840
epoch: 0 Loss: 1.0213
epoch: 0 Loss: 1.9748
epoch: 0 Loss: 1.5413
epoch: 0 Loss: 2.3123
epoch: 0 Loss: 1.8131
epoch: 0 Loss: 1.3540
epoch: 0 Loss: 1.7792
epoch: 0 Loss: 0.8160
epoch: 0 Loss: 1.4878
epoch: 0 Loss: 2.0312
epoch: 0 Loss: 1.6843
epoch: 0 Loss: 1.7600
epoch: 0 Loss: 1.0868
epoch: 0 Loss: 1.5406
epoch: 0 Loss: 1.4920
epoch: 0 Loss: 1.9743
epoch: 0 Loss: 2.0738
epoch: 0 Loss: 1.7415
epoch: 0 Loss: 1.0732
epoch: 0 Loss: 1.2194
epoch: 0 L

In [28]:
start_idx = len(train_ds)

targets = []
all_preds = []
with torch.no_grad():
    model.eval()
    for i, (imgs, labels) in enumerate(eval_loader):
        imgs = imgs.to(device)
        labels = labels.to(device)

        y_pred = model(imgs).argmax(dim=-1)
        targets.extend(labels.cpu().numpy())
        all_preds.extend(y_pred.cpu().numpy())
#         print(y_pred, labels)
#         tensor([8, 2, 2, 2, 8, 2, 8, 7], device='cuda:0') tensor([11,  2,  1,  2,  8,  2,  8,  7], device='cuda:0')

print('done!')

done!


In [27]:
print("Accuracy: {}".format( accuracy_score(targets, all_preds)))
print("F1 Loss: {:.4f}".format( np.mean(f1_score(targets, all_preds, average=None)) ))

Accuracy: 0.7021164021164021
F1 Loss: 0.4136


## 2. Test Dataset 정의

In [29]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

## 3. Inference

In [38]:
torch.save(model, os.path.join(model_dir, "resnet34_crossentropy_adam.pt"))

In [42]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
model = torch.load(os.path.join(model_dir, "resnet34_crossentropy_adam.pt"))
model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
# submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
submission.to_csv(os.path.join(test_dir, 'resnet34_crossentropy_adam.csv'), index=False)
print('test inference is done!')